In [2]:
import jax 
import scipy
import jax.numpy as jnp
import diffrax as dfx
import optax as opx

In [3]:
from simulate import generate_y0, simulate_pdu, measure_pdu
from pdu_rhs import V_RATIO
import numpy as np

data_ICs = jnp.array([80, 0, 0, 0]) 
params = jnp.array([.5, .4, .8, .9, .6])
y0 = generate_y0(params, data_ICs)

t0 = 0.0
t1 = 10.0*3600
dt = 1e-2

params = jnp.array([ .5, .4, .8, 1e8, 40, \
1e3, -3000, 100, 50, 200, 140, \
1e4, -5000, 40, 10, 100, 70, \
1e9,  -10000, 50, 85, \
1e9, -10000, 10, 100, \
-5, -6, \
-4, 1.2*V_RATIO])

t_out, y_out = measure_pdu(simulate_pdu(params, y0, t0, t1, dt))
y_out_sim = y_out + np.random.normal(0, 1, y_out.shape)

In [16]:
np.mean((y_out_sim - y_out)**2)

Array(1.03745622, dtype=float64)

In [20]:
from objective import construct_loss, parameterize_loss

options = {"loss_fn": "L2",
           "t_weight": "inv_stepsize",
           "weight": "max_relative_scale",
           }

loss_fn = construct_loss(options)
loss_fn = parameterize_loss(y_out, t_out, loss_fn)

loss_fn(y_out, y_out_sim)

Array(2119.10429785, dtype=float64)